In [1]:
from sklearn.pipeline import Pipeline
import pandas as pd
import csv

# for custom functions
import os
import sys
# import sys; sys.path.insert(0, '..')
# from lib.custom_functions import * 

In [2]:
import pandas as pd
from datetime import date

def mean(s: pd.Series):
    return s.mean()

def median(s: pd.Series):
    return s.median()

def drop_features(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    df = df.drop(columns, axis=1)
    return df

def bool_to_binary(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    for column in columns:
        local_kwargs = {column : lambda df: df[column].astype(str).str.replace('t', '1').replace('f', '0').replace('True', '1').replace('False', '0').astype(int)}

        df = (
            df
            .assign(**local_kwargs)
        )
        
        return df
    
def process_neighbourhoods(df: pd.DataFrame, quantile: float=0.75) -> pd.DataFrame:
    if os.path.isfile('data/neighbourhoods_to_keep.csv'):
        neighbourhoods_to_keep = pd.read_csv('data/neighbourhoods_too_keep.csv')
        
    else:
        yf = df.groupby(by=['neighbourhood']).id.count().sort_values(ascending=False).reset_index()
        neighbourhoods_to_keep = (
            pd.DataFrame(yf.loc[yf.id > yf.id.quantile(q=quantile), 'neighbourhood'])
            .assign(neighbourhood_to_keep=lambda df: df['neighbourhood'])
            .drop('neighbourhood', axis=1)
        )
        neighbourhoods_to_keep.to_csv('data/neighbourhoods_to_keep.csv')
        
    df = (
        df
        .merge(neighbourhoods_to_keep, how='left', left_on='neighbourhood', right_on='neighbourhood_to_keep')
    )
    
    return df
        
def fill_na(df: pd.DataFrame, replacements: dict) -> pd.DataFrame:
    for k, v in replacements.items():

        if callable(v):
            value = v(df[k])
        else:
            value = v
        
        local_kwargs = {k : lambda df: df[k].fillna(value)}

        df = (
            df
            .assign(**local_kwargs)
        )
    return df

def one_hot_encoder(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    df = pd.get_dummies(
        df, columns=columns, prefix='', prefix_sep=''
    )

    return df

def process_host_response_rate(df: pd.DataFrame) -> pd.DataFrame:
    df = (
        df
        .assign(host_response_rate=lambda df: df.host_response_rate.str.replace('%', '').astype(float))
        .assign(host_response_class=lambda df: pd.cut(df.host_response_rate, 
                                                    bins=[0, 75, 80, 85, 90, 95, 100.00], 
                                                    labels=['very_low',
                                                            'low',
                                                            'medium',
                                                            'high',
                                                            'very_high',
                                                            'best'], 
                                                    include_lowest=True))
        .assign(host_response_class=lambda df: df.host_response_class.cat.add_categories('other').fillna('other'))
        .drop('host_response_rate', axis=1)
    )
    
    return df

def process_review_dates(df: pd.DataFrame) -> pd.DataFrame:
    df = (
    df
        .assign(first_review=lambda df: pd.to_datetime(df.first_review))
        .assign(last_review=lambda df: pd.to_datetime(df.last_review))
        .assign(host_since=lambda df: pd.to_datetime(df.host_since))
        .assign(today=[date.today()]*len(df.index))
        .assign(today=lambda df: pd.to_datetime(df.today))
        .assign(days_since_first_review=lambda df: (df.today - df.first_review).dt.days)
        .assign(days_since_last_review=lambda df: (df.today - df.last_review).dt.days)
        .assign(days_host_since=lambda df: (df.today - df.host_since).dt.days)
        .assign(first_review_class=lambda df: pd.cut(df.days_since_first_review, 
                                                    bins=[0, 1095, 1280, 1460, 1645, 1830, max(df.days_since_first_review)], 
                                                    labels=['0y-3y',
                                                            '3y-3y6m',
                                                            '3y6m-4y',
                                                            '4y-4y6m',
                                                            '4y6m-5y',
                                                            '5y'], 
                                                    include_lowest=True))
        .assign(last_review_class=lambda df: pd.cut(df.days_since_last_review, 
                                                    bins=[0, 1095, 1220, 1460, 1645, 1830, max(df.days_since_first_review)], 
                                                    labels=['0y-3y',
                                                            '3y-3y6m',
                                                            '3y6m-4y',
                                                            '4y-4y6m',
                                                            '4y6m-5y',
                                                            '5y'], 
                                                    include_lowest=True))
        .assign(first_review_class=lambda df: df.first_review_class.cat.add_categories('other').fillna('other'))
        .assign(last_review_class=lambda df: df.last_review_class.cat.add_categories('other').fillna('other'))
        .assign(days_host_since=lambda df: df.days_host_since.fillna(1))
        .drop(['first_review', 'last_review', 'today', 'days_since_first_review', 'days_since_last_review', 'host_since'], axis=1)
    )
    
    return df

def process_amenities(df: pd.DataFrame) -> pd.DataFrame:
    amenities_to_keep = []
    
    if os.path.isfile('data/amenities_to_keep'):
        print('process_amenities -> reading amenities_to_keep from file')
        with open('data/amenities_to_keep', 'r') as f: 
            reader = csv.reader(f)

            for row in reader:
                amenities_to_keep = row
            
    if not amenities_to_keep:
        print('process_amenities -> processing amenities_to_keep')
        amenities = [x.strip() for x in ", ".join(list(df.amenities)).replace('{', '').replace('}', '').replace('"', '').replace('TV', '').split(',')]

        amenities_counter = {}
        for amenity in amenities:
            if amenity in amenities_counter.keys():
                amenities_counter[amenity] += 1
            else:
                amenities_counter[amenity] = 1

        amenities_counter_sorted = {k: v for k, v in sorted(amenities_counter.items(), key=lambda item: item[1])}

        amenities_to_keep = []
        records_count = (len(amenities_counter_sorted) + 0.001)
        for i, e in (enumerate(amenities_counter_sorted.items())):
            if (i*100.0/records_count > 20 or i*100.0/records_count < 80) \
            and (e[1] > 2) \
            and ("translation missing" not in e[0]) \
            and (e[0] != ''):
                amenities_to_keep.append(e[0])
                
        with open('data/amenities_to_keep', 'w') as f: 
            print('process_amenities -> writing amenities_to_keep to file')
            write = csv.writer(f) 
            write.writerow(amenities_to_keep)
                
        
            
    for amenity in amenities_to_keep:
        df[amenity] = df.apply(lambda row: 1 if amenity in row.amenities else 0, axis=1)
        
    df = df.drop(['amenities'], axis=1)
    
    return df

def process_property_type(df: pd.DataFrame, column: str, replacements: dict) -> pd.DataFrame:
    local_kwargs = {column : lambda df: df[column].replace(replacements)}

    df = (
        df
        .assign(**local_kwargs)
    )
    
    df.loc[~df.property_type.isin(['House', 'Apartment']), 'property_type'] = 'other'

    return df

In [3]:
df_train = pd.read_csv('data/train.csv')
df_train = (
    df_train
        .pipe(drop_features, columns=['latitude', 'longitude', 'name', 'thumbnail_url', 'zipcode', 'host_has_profile_pic', 'description'])
        .pipe(fill_na, replacements={'neighbourhood': 'other'})
        .pipe(process_neighbourhoods, quantile=0.8)
        .pipe(drop_features, columns=['neighbourhood'])
        .pipe(fill_na, replacements={'number_of_reviews': 0,
                                     'bedrooms': 0,
                                     'review_scores_rating': mean,
                                     'beds': median,
                                     'bathrooms': median,
                                     'accommodates': median,
                                     'neighbourhood_to_keep': 'other'})
        .pipe(bool_to_binary, columns=['cleaning_fee', 'host_identity_verified', 'instant_bookable'])
        .pipe(process_property_type, column='property_type', replacements={'Townhouse': 'House',
                                                                           'Bungalow': 'House',
                                                                           'Cottage': 'House',
                                                                           'Villa': 'House',
                                                                           'Tiny house': 'House',
                                                                           'Chalet': 'House',
                                                                           'Serviced apartment': 'Apartment',
                                                                           'Loft': 'Apartment',
                                                                           'Condominium': 'Apartment'})
        .pipe(process_host_response_rate)
        .pipe(process_review_dates)
        .pipe(process_amenities)
        .pipe(one_hot_encoder, 
              columns=['instant_bookable', 
                       'host_identity_verified', 
                       'city', 
                       'cleaning_fee',
                       'cancellation_policy', 
                       'bed_type', 
                       'room_type', 
                       'host_response_class', 
                       'neighbourhood', 
                       'property_type',
                       'neighbourhood_to_keep'])

)

df_train.to_csv('data/train_processed.csv')

process_amenities -> reading amenities_to_keep from file


KeyError: "['neighbourhood'] not in index"

In [ ]:
df_train_processed = pd.read_csv('data/train_processed.csv')
df_train_processed.head()